In [12]:
import pandas as pd
import numpy as np

# Helper functions
def get_joint_xyz(df, joint_id):
    return df[[f'x{joint_id}', f'y{joint_id}', f'z{joint_id}']].values

def compute_angle(a, b, c):
    ba = a - b
    bc = c - b
    cosine = np.einsum('ij,ij->i', ba, bc) / (
        np.linalg.norm(ba, axis=1) * np.linalg.norm(bc, axis=1) + 1e-8
    )
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

In [13]:
df = pd.read_csv(r"C:\Users\rayaa\OneDrive\Desktop\Sproj_Shoulder\data\benchpress_coords.csv")


In [14]:
df_coords = df[[col for col in df.columns if col[0] in ['x','y','z'] and not col.startswith('v')]].copy()
df_coords['class'] = df['class']

In [15]:
# Compute features
features = pd.DataFrame()
# Joint angles
features['left_elbow_angle'] = compute_angle(get_joint_xyz(df_coords,15), get_joint_xyz(df_coords,13), get_joint_xyz(df_coords,11))
features['right_elbow_angle'] = compute_angle(get_joint_xyz(df_coords,16), get_joint_xyz(df_coords,14), get_joint_xyz(df_coords,12))
features['left_shoulder_angle'] = compute_angle(get_joint_xyz(df_coords,13), get_joint_xyz(df_coords,11), get_joint_xyz(df_coords,23))
features['right_shoulder_angle'] = compute_angle(get_joint_xyz(df_coords,14), get_joint_xyz(df_coords,12), get_joint_xyz(df_coords,24))
features['left_knee_angle'] = compute_angle(get_joint_xyz(df_coords,27), get_joint_xyz(df_coords,25), get_joint_xyz(df_coords,23))
features['right_knee_angle'] = compute_angle(get_joint_xyz(df_coords,28), get_joint_xyz(df_coords,26), get_joint_xyz(df_coords,24))
# Form metrics
features['left_elbow_flare'] = np.abs(df_coords['x13']-df_coords['x11']) - np.abs(df_coords['x15']-df_coords['x11'])
features['right_elbow_flare'] = np.abs(df_coords['x14']-df_coords['x12']) - np.abs(df_coords['x16']-df_coords['x12'])
features['left_wrist_stack_diff'] = np.sqrt((df_coords['y15']-df_coords['y13'])**2 + (df_coords['z15']-df_coords['z13'])**2)
features['right_wrist_stack_diff'] = np.sqrt((df_coords['y16']-df_coords['y14'])**2 + (df_coords['z16']-df_coords['z14'])**2)
features['shoulder_diff_3d'] = np.sqrt((df_coords['y12']-df_coords['y11'])**2 + (df_coords['z12']-df_coords['z11'])**2)
features['elbow_angle_asym'] = np.abs(features['left_elbow_angle'] - features['right_elbow_angle'])
features['shoulder_angle_asym'] = np.abs(features['left_shoulder_angle'] - features['right_shoulder_angle'])
# Attach class
features['class'] = df_coords['class']
features.head()

,left_elbow_angle,right_elbow_angle,left_shoulder_angle,right_shoulder_angle,left_knee_angle,right_knee_angle,left_elbow_flare,right_elbow_flare,left_wrist_stack_diff,right_wrist_stack_diff,shoulder_diff_3d,elbow_angle_asym,shoulder_angle_asym,class
0,155.829063,161.920910,23.171719,125.226992,127.353077,36.806798,0.007993,-0.034835,0.343165,0.108309,0.353820,6.091848,102.055274,b_correct_up
1,154.619283,138.610598,27.678764,103.665383,144.719684,44.077452,-0.039567,-0.083237,0.289852,0.028476,0.313675,16.008685,75.986619,b_correct_down
2,163.594116,147.676605,21.102228,120.954584,109.685019,39.804694,-0.053491,-0.020218,0.292091,0.130633,0.235194,15.917511,99.852356,b_correct_down
3,155.346918,112.358555,27.230365,94.812442,126.956354,46.305354,-0.029823,-0.108165,0.326045,0.099072,0.319935,42.988363,67.582077,b_correct_down
4,154.454148,145.625799,25.099893,129.222059,126.102640,39.764947,-0.001489,-0.018972,0.340309,0.095232,0.367216,8.828349,104.122166,b_correct_up


In [17]:
# One-hot encode classes and compute correlations
class_dummies = pd.get_dummies(features['class'], prefix='class')
corr_input = pd.concat([features.drop(columns=['class']), class_dummies], axis=1)
correlations = corr_input.corr().loc[features.columns.drop('class'), class_dummies.columns]
correlations

print(correlations)

                        class_b_arms_spread_down  class_b_arms_spread_up  \
left_elbow_angle                        0.343420                0.017688   
right_elbow_angle                      -0.001695               -0.066005   
left_shoulder_angle                    -0.225802               -0.080070   
right_shoulder_angle                    0.183221                0.239087   
left_knee_angle                        -0.099903                0.157840   
right_knee_angle                        0.158858                0.305265   
left_elbow_flare                       -0.263995               -0.082080   
right_elbow_flare                      -0.350348               -0.038645   
left_wrist_stack_diff                   0.079007               -0.089561   
right_wrist_stack_diff                 -0.121109                0.018588   
shoulder_diff_3d                        0.029502               -0.004153   
elbow_angle_asym                        0.181493                0.006100   
shoulder_ang

In [18]:

# Save to CSV
features.to_csv("benchpress_angles.csv", index=False)
